### Forked from [Logicmn/pyx](https://github.com/Logicmn/pyx)

In [ ]:
!pip install yahoo_finance, yahoo_finance_api2, yfinance, sqlalchemy

In [1]:
import datetime
# from yahoo_finance import Share
# from yahoo_finance_api2.share import Share, PERIOD_TYPE_DAY, FREQUENCY_TYPE_MINUTE, FREQUENCY_TYPE_DAY, FREQUENCY_TYPE_HOUR
from yfinance import Ticker

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Sequence, MetaData, create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_db.db', echo=True) # Link the database to the SQLAlchemy engine
Session = sessionmaker(bind=engine)
Base = declarative_base()
metadata = MetaData()
session = Session()

class Wallet(Base): # Create 'wallets' table
    __tablename__ = 'wallets'

    id = Column(Integer, Sequence('wallet_id_seq'), primary_key=True)
    name = Column(String)
    balance = Column(Integer)

    def __repr__(self):
        return "<Wallet(name='%s', balance='%s')>" % (self.name, self.balance)

class Transaction(Base): # Create 'transactions' table
    __tablename__ = 'transactions'

    id = Column(Integer, Sequence('transaction_id_seq'), primary_key=True)
    stock = Column(String(50))
    symbol = Column(String(50))
    buy_or_sell = Column(String(50))
    price = Column(Integer())
    ema = Column(Integer())
    shares = Column(Integer())
    time = Column(String(50))

    def __repr__(self):
        return "<Transaction(stock='%s', symbol='%s', buy_or_sell='%s', price='%s', ema='%s', shares='%s', time='%s')>"\
               % (self.stock, self.symbol, self.buy_or_sell, self.price, self.ema, self.shares, self.time)

class Strategy(object): # Create the algorithm PYX will use to trade with
    def __init__(self, equity):
        self.equity = equity

    def getEquity(self):
        return self.equity

    def calcEMA(self, close_price, prev_ema): # Calculate the exponential moving average
        multiplier = 2 / (50 + 1)
        ema = (close_price - prev_ema) * multiplier + prev_ema
        return ema

    def calcUpper(self, ema): # Calculate the upper Bollinger band
        upper_band = ema * (1 + .02)
        return upper_band

    def calcLower(self, ema): # Calculate the lower Bollinger band
        lower_band = ema * (1 - .02)
        return lower_band

def enter_position(mean_reversion, stock): # Buy shares of a stock
    close_price, prev_ema, ema, lower_band, upper_band, purchase_query = calculations(mean_reversion, stock)
    if purchase_query != None:
        purchase = purchase_query[0]
    else:
        purchase = 'sell'
    if float(stock.get_price()) <= lower_band and purchase != 'buy': # Buy shares if the last purchase was a sell
        print('buy')
        new_transaction = Transaction(stock=stock.get_name(), symbol=stock.symbol, buy_or_sell='buy',
                                      price=stock.get_price(),
                                      ema=ema, shares='100', time=datetime.datetime.now())
        session.add(new_transaction)
        session.commit()
        balance, new_funds = calc_wallet()
        new_bal = balance[0] - new_funds # Subtract amount spent from the balance in wallet
        primary_wallet = Wallet(name='Primary Wallet', balance=new_bal) # Re-create wallet with new balance
        session.add(primary_wallet)
        session.commit()


def exit_position(mean_reversion, stock): # Sell shares of a stock
    close_price, prev_ema, ema, lower_band, upper_band, purchase_query = calculations(mean_reversion, stock)
    if purchase_query != None:
        purchase = purchase_query[0]
    elif purchase_query == None:
        purchase = 'sell'
    else:
        purchase = 'buy'
    if float(stock.get_price()) >= upper_band and purchase != 'sell': # Sell shares if the last purchase was a buy
        print('sell')
        new_transaction = Transaction(stock=stock.get_name(), symbol=stock.symbol, buy_or_sell='sell',
                                      price=stock.get_price(),
                                      ema=ema, shares='100', time=datetime.datetime.now())
        session.add(new_transaction)
        session.commit()
        balance, new_funds = calc_wallet()
        new_bal = balance[0] + new_funds # Add amount gained to the balance in wallet
        primary_wallet = Wallet(name='Primary Wallet', balance=new_bal) # Re-create wallet with new balance
        session.add(primary_wallet)
        session.commit()

def calculations(mean_reversion, stock):
    close_price = float(stock.get_prev_close())# Calculate yesterdays close price
    prev_ema = float(stock.get_50day_moving_avg()) # Calculate the previous EMA
    ema = mean_reversion.calcEMA(close_price, prev_ema) # Calculate the EMA
    lower_band, upper_band= float(mean_reversion.calcLower(ema)), float(mean_reversion.calcUpper(ema)) # Calculate the bands
    print("-------------------------")
    purchase_query = session.query(Transaction.buy_or_sell).order_by(
                     Transaction.id.desc()).first()  # Find out whether the latest purchase was a buy/sell
    return close_price, prev_ema, ema, lower_band, upper_band, purchase_query

def calc_wallet():
    new_price = session.query(Transaction.price).order_by(Transaction.id.desc()).first() # Grab the bought price
    new_shares = session.query(Transaction.shares).order_by(Transaction.id.desc()).first() # Grab how many shares were bought
    new_funds = new_price[0] * new_shares[0] # Calculate the money spent
    balance = session.query(Wallet.balance).one() # Grab the current balance
    current_bal = session.query(Wallet).one()
    session.delete(current_bal) # Delete the wallet
    session.commit()
    return balance, new_funds

In [20]:
class Share(Ticker):
    def __init__(self, symbol):
        self.symbol = symbol
        
    def get_price(self):
        return 110.0
    
    def get_name(self):
        return self.symbol
    
    def get_prev_close(self):
        return 128.0
    
    def get_50day_moving_avg(self):
        return 135.0
        

In [21]:
stock = Share("AAPL")

In [22]:
# import pandas as pd

# if stock.get_historical(PERIOD_TYPE_DAY, 1, FREQUENCY_TYPE_DAY, 5) is None:
#     print("Error : {} has no price history".format(symbol))
#     exit(1)
    
# out = pd.DataFrame.from_dict(stock.get_historical(PERIOD_TYPE_DAY, 1, FREQUENCY_TYPE_HOUR, 1))
# out["timestamp2"] = out.timestamp.apply(lambda ts: datetime.datetime.fromtimestamp(ts/1e3))
# display(out)

In [23]:
Base.metadata.create_all(engine)
session.commit()

b = session.query(Wallet.balance).first() # Check if there is already a wallet

if b == None:
    primary_wallet = Wallet(name='Primary Wallet', balance=100000) # Create the wallet with a balance of $100,000
    session.add(primary_wallet)
    session.commit()

mean_reversion = Strategy(stock.symbol) # Run the EMA, and Bollinger band calculations
enter_position(mean_reversion, stock) # Buy stock if applicable
exit_position(mean_reversion, stock) # Sell stock if applicable
session.commit()

2021-02-25 15:19:22,444 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("wallets")
2021-02-25 15:19:22,446 INFO sqlalchemy.engine.base.Engine ()
2021-02-25 15:19:22,448 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("transactions")
2021-02-25 15:19:22,450 INFO sqlalchemy.engine.base.Engine ()
2021-02-25 15:19:22,453 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-25 15:19:22,454 INFO sqlalchemy.engine.base.Engine SELECT wallets.balance AS wallets_balance 
FROM wallets
 LIMIT ? OFFSET ?
2021-02-25 15:19:22,456 INFO sqlalchemy.engine.base.Engine (1, 0)
-------------------------
2021-02-25 15:19:22,460 INFO sqlalchemy.engine.base.Engine SELECT transactions.buy_or_sell AS transactions_buy_or_sell 
FROM transactions ORDER BY transactions.id DESC
 LIMIT ? OFFSET ?
2021-02-25 15:19:22,462 INFO sqlalchemy.engine.base.Engine (1, 0)
buy
2021-02-25 15:19:22,465 INFO sqlalchemy.engine.base.Engine INSERT INTO transactions (stock, symbol, buy_or_sell, price, ema, sha

In [24]:
# loading in modules
import sqlite3
import pandas as pd

# creating file path
dbfile = 'new_db.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)

wallets_df = pd.read_sql_query('SELECT * FROM wallets', con)
trans_df = pd.read_sql_query('SELECT * FROM transactions', con)

# Be sure to close the connection
con.close()

[('wallets',), ('transactions',)]


In [25]:
wallets_df

,id,name,balance
0,1,Primary Wallet,93500


In [26]:
trans_df

,id,stock,symbol,buy_or_sell,price,ema,shares,time
0,1,AAPL,AAPL,buy,125,134.72549,100,2021-02-25 15:07:53.180044
1,2,AAPL,AAPL,sell,170,134.72549,100,2021-02-25 15:13:08.169562
2,3,AAPL,AAPL,buy,110,134.72549,100,2021-02-25 15:19:22.464680
